举例1

In [6]:
from langchain_core.tools import tool, StructuredTool


@tool
def add_number(a:int,b:int)->int:
    """计算两个整数的和"""
    return a+b

print(f"name = {add_number.name}")
print(f"args = {add_number.args}")
print(f"description = {add_number.description}")
print(f"return_direct = {add_number.return_direct}")


name = add_number
args = {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
description = 计算两个整数的和
return_direct = False


举例2


In [17]:
from langchain_core.tools import tool
from pydantic import Field
from pydantic import BaseModel

class FieldInfo(BaseModel):
    a:int = Field(description="第一个整型参数")
    b:int = Field(description="第二个整型参数")
@tool("add_two_number",return_direct=True,args_schema=FieldInfo)
def add_number(a:int,b:int)->int:
    """计算两个整数的和"""
    return a+b

print(f"name = {add_number.name}")
print(f"args = {add_number.args}")
print(f"description = {add_number.description}")
print(f"return_direct = {add_number.return_direct}")
# 调用工具
add_number.invoke({"a":1,"b":2})

name = add_two_number
args = {'a': {'description': '第一个整型参数', 'title': 'A', 'type': 'integer'}, 'b': {'description': '第二个整型参数', 'title': 'B', 'type': 'integer'}}
description = 计算两个整数的和
return_direct = True


3

2.StructuredTool的from_function()

举例1


In [21]:
from langchain_core.tools.structured import StructuredTool
class FieldInfo(BaseModel):
    query:str = Field(description="第一个整型参数")

def search_google(query:str):
    return "最后查询的结果"
# 定义一个工具
search01 = StructuredTool.from_function(
    func=search_google,
    name="Search Google",
    description="查询google搜索引擎的结果，并将结果返回",
    return_direct = True,
    args_schema=FieldInfo,

)
print(f"name = {search01.name}")
print(f"args = {search01.args}")
print(f"description = {search01.description}")
search01.invoke({"query":"中美AI发展现状"})

name = Search Google
args = {'query': {'description': '第一个整型参数', 'title': 'Query', 'type': 'string'}}
description = 查询google搜索引擎的结果，并将结果返回


'最后查询的结果'

大模型分析工具的调用

In [50]:
#1.导入相关依赖
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
import os
import dotenv
from langchain_openai import ChatOpenAI
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
# 定义LLM模型
chat_model =ChatOpenAI(model="gpt-4o-mini",temperature=0)
# 2.获取工具列表
tools = [MoveFileTool()]
functions = [convert_to_openai_function(t) for t in tools]
# 3.获取消息列表
message = [HumanMessage(content="将当前目录下的文件a.txt移动到\\Users\\zz\\Desktop")]
# 调用大模型
response = chat_model.invoke(
    input=message,
    functions=functions,
)
print(response)

content='' additional_kwargs={'function_call': {'arguments': '{"source_path":"a.txt","destination_path":"\\\\Users\\\\zz\\\\Desktop\\\\a.txt"}', 'name': 'move_file'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 85, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f97eff32c5', 'finish_reason': 'function_call', 'logprobs': None} id='run-8b668a12-02b6-4457-ad3d-67400ce979f5-0' usage_metadata={'input_tokens': 85, 'output_tokens': 30, 'total_tokens': 115}


如何调用具体大模型分析出来的工具

In [51]:
import json

tool_args = json.loads(response.additional_kwargs["function_call"]["arguments"])
if "move_file" in response.additional_kwargs["function_call"]["name"]:
    tool = MoveFileTool()
    result = tool.run(tool_args)
    print("工具执行的结果",result)

工具执行的结果 Error: no such file or directory a.txt
